**LUYỆN TẬP THỰC HÀNH TENSORFLOW**

[DAY 2] TEXT AND IMAGE PROCESSING

In [3]:
!pip install tensorflow==2.3.0
!pip install tensorflow_datasets==3.2.1
!pip install Pillow==7.2.0

     |████████████████████████████████| 320.4 MB 47 kB/s 
     |████████████████████████████████| 459 kB 50.1 MB/s 
     |████████████████████████████████| 2.9 MB 52.3 MB/s 
     |████████████████████████████████| 20.1 MB 103.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:

     |████████████████████████████████| 3.4 MB 7.2 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
     |████████████████████████████████| 2.2 MB 5.8 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import urllib
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil

In [2]:
print(tf.__version__)

2.3.0


1. Image classification

Bộ dữ liệu Người hay ngựa (Horse or human) được tải về sẵn như dưới

In [3]:
_TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
_TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
local_zip = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/horse-or-human/')
zip_ref.close()
urllib.request.urlretrieve(_TEST_URL, 'testdata.zip')
local_zip = 'testdata.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/testdata/')
zip_ref.close()

In [4]:
dir_traindata = '/content/tmp/horse-or-human/'
dir_testdata = '/content/tmp/testdata/'

dir = '/content/tmp'
train_folder = 'train'
test_folder = 'test'
os.mkdir(os.path.join(dir, train_folder))
os.mkdir(os.path.join(dir, test_folder))

In [5]:
dst_train_folder = '/content/tmp/train'
dst_test_folder = '/content/tmp/test'
folders = os.listdir(dir)
for folder1 in folders:
  if folder1 != 'train' and folder1 != 'test':
    path_folder1 = os.path.join(dir, folder1)
    
    for folder2 in os.listdir(path_folder1):
      path_folder2 = os.path.join(path_folder1, folder2)

      for filename in os.listdir(path_folder2):
        if folder1=='horse-or-human':
          shutil.copy(os.path.join(path_folder2, filename), dst_train_folder)
        else:
          shutil.copy(os.path.join(path_folder2, filename), dst_test_folder)

In [6]:
import pandas as pd

filenames = os.listdir('/content/tmp/train')
categories = []
for filename in filenames:
    if filename.find('human') != -1:
        categories.append(str(1))
    else:
        categories.append(str(0))

df_train = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [7]:
filenames = os.listdir('/content/tmp/test')
categories = []
for filename in filenames:  
  if filename.find('human') != -1:
    categories.append(str(1))
  else:
    categories.append(str(0))

df_test = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255)

valid_datagen = ImageDataGenerator(
    rescale=1./255)

In [9]:
img_size = (300, 300)
batch_size = 64
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    "/content/tmp/train", 
    x_col='filename',
    y_col='category',
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    target_size=img_size,
    class_mode='binary',
    batch_size=batch_size
 )

valid_generator = valid_datagen.flow_from_dataframe( 
    df_test,
    "/content/tmp/test", 
    x_col='filename',
    y_col='category',
    target_size=img_size,
    class_mode='binary',
    batch_size=batch_size
 )

Found 1027 validated image filenames belonging to 2 classes.
Found 256 validated image filenames belonging to 2 classes.


In [10]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (64, 300, 300, 3)
labels batch shape: (64,)


In [11]:
train_data, train_label = train_generator.next()

In [14]:
test_data, test_label = valid_generator.next()

In [13]:
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False)
for layer in base_model.layers:
  layer.trainable=False

model = tf.keras.Sequential([
                             base_model,
                             keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dropout(rate=0.2),
                             keras.layers.Dense(180, activation='relu'),
                             keras.layers.Dropout(rate=0.2),
                             keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=['accuracy'])

87916544/87910968 [==============================] - 0s 0us/step


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=2, mode='max')

In [16]:
model.fit_generator(
    train_generator,
    epochs=10,
    verbose=2,
    callbacks=[lr_reduce])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
17/17 - 21s - loss: 0.5543 - accuracy: 0.8841
Epoch 2/10
17/17 - 22s - loss: 0.5061 - accuracy: 0.9854
Epoch 3/10
17/17 - 23s - loss: 0.5031 - accuracy: 0.9893
Epoch 4/10
17/17 - 23s - loss: 0.4996 - accuracy: 0.9981
Epoch 5/10
17/17 - 22s - loss: 0.5002 - accuracy: 0.9961
Epoch 6/10
17/17 - 22s - loss: 0.5016 - accuracy: 0.9942
Epoch 7/10
17/17 - 23s - loss: 0.4994 - accuracy: 0.9981
Epoch 8/10
17/17 - 26s - loss: 0.4997 - accuracy: 0.9971
Epoch 9/10
17/17 - 22s - loss: 0.5003 - accuracy: 0.9971
Epoch 10/10
17/17 - 22s - loss: 0.4997 - accuracy: 0.9981


In [19]:
test_loss, test_acc = model.evaluate(valid_generator, verbose=2)

4/4 - 2s - loss: 0.5033 - accuracy: 1.0000


In [20]:
print(f"Test loss: {test_loss}; Test accuracy: {round(test_acc*100, 2)}%")

Test loss: 0.5033109784126282; Test accuracy: 100.0%


Dữ liệu ảnh Người - ngựa đã được lưu vào các thư mục: dir_train, dir_test.
Hãy load dữ liệu này thành dataset và huấn luyện một mô hình phân loại người - ngựa


2. Text classification

Bộ dữ liệu Sarcasm được tải về như dưới, hãy xử lý huấn luyện một mô hình phân loại văn bản là mỉa mai hay không.

In [60]:
import json

url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = []
labels = []

# YOUR CODE HERE
with open('sarcasm.json', 'r') as f:
    data = json.load(f)

for d in data:
    sentences.append(d['headline'])
    labels.append(d['is_sarcastic'])

import numpy as np

sentences = []
labels = []
for d in data:
  sentences.append(d['headline'])
  labels.append(d['is_sarcastic'])

In [73]:
token = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token = oov_tok)
token.fit_on_texts(sentences)

sequences = token.texts_to_sequences(sentences)

data = sequences.copy()
labels = np.array(labels)

In [74]:
data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=max_length, padding='post', truncating='post')

In [75]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", 
                          patience = 7, 
                          verbose = 1,  
                          restore_best_weights = True, 
                          mode = 'max')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_accuracy", 
                              factor = .4642,
                              patience = 3,
                              verbose = 1, 
                              min_delta = 0.001,
                              mode = 'max')

In [76]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, recurrent_dropout=0.5, dropout=0.5)),
                             tf.keras.layers.Dense(24, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=30,
                    validation_data=(x_val, y_val))

Epoch 1/30
377/677 [===============>..............] - ETA: 3:56 - loss: 0.5086 - accuracy: 0.7322